In [ ]:
import numpy as np
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from collections import Counter
warnings.filterwarnings('ignore')

from snowflake.snowpark.context import get_active_session
session = get_active_session()

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 150)
pd.set_option('display.float_format', '{:.2f}'.format)

# --- Load data from the four specified tables ---
df_customers = session.table("BANKDATA.REFINED.REFINED_BANK_CUSTOMERS").to_pandas()
df_assets = session.table("BANKDATA.REFINED.REFINED_BANK_CUSTOMER_ASSETS").to_pandas()
df_loans = session.table("BANKDATA.REFINED.REFINED_BANK_LOANS").to_pandas()
df_payments = session.table("BANKDATA.REFINED.REFINED_BANK_PAYMENTS").to_pandas()

# --- Pre-processing for loan-centric analysis ---

df_customers

In [ ]:
# Remove commas from Phone_Number and Account_Number columns
df_customers['PHONE_NUMBER'] = df_customers['PHONE_NUMBER'].astype(str).str.replace(',', '', regex=False)
df_customers['INCOME(MONTH)'] = df_customers['INCOME(MONTH)'].astype(str).str.replace(',', '', regex=False)


In [ ]:
df_customers

In [ ]:
df_customers['EMAIL'] = df_customers.apply(
    lambda row: f"{row['FIRST_NAME'].lower()}.{row['LAST_NAME'].lower()}@gmail.com" 
    if pd.isnull(row['EMAIL']) else row['EMAIL'], axis=1
)

In [ ]:
Tables = [df_assets, df_customers, df_loans, df_payments]
table_names = ["df_assets", "df_customers", "df_loans", "df_payments"] # Added for clearer output

for i, table in enumerate(Tables):
    print(f"--- Analysis for {table_names[i]} ---") # Clearly indicate which table is being analyzed
    print(f"Shape: {table.shape}")
    print(f"Size: {table.size}")
    print("\nTable Information:")
    table.info() # Call .info() directly, it prints to console and returns None
    print("\nMissing values per column:")
    print(table.isnull().sum())           # Handle missing values
    print("\nFirst 5 rows:")
    print(table.head())
    print("-" * 50) # Separator for clarity between tables
    print("\n")


In [ ]:
# Step 1: Start with the base loan data
loan_data = df_loans.copy()

# Step 2: Merge customer details (including additional fields)
loan_data = loan_data.merge(
    df_customers[[
        'CUSTOMER_ID', 'FIRST_NAME', 'LAST_NAME', 'ACCOUNT_NUMBER', 'PAN_NUMBER',
        'INCOME(MONTH)', 'CIBIL_SCORE', 'AGE', 'GENDER', 'MARITAL_STATUS', 'EMPLOYMENT_STATUS'
    ]],
    on='CUSTOMER_ID',
    how='left'
)

# Step 3: Prepare payment data
df_payments['TOTAL_PAID'] = df_payments['AMOUNT_PAID'].fillna(0)

# Step 4: Aggregate payment info per loan
payments_agg = df_payments.groupby('LOAN_ID').agg(
    TOTAL_PAID=('TOTAL_PAID', 'sum'),
    NUM_PAYMENTS=('LOAN_ID', 'count')
).reset_index()

# Step 5: Merge aggregated payments into loan_data
loan_data = loan_data.merge(payments_agg, on='LOAN_ID', how='left')

# Step 6: Get most recent STATUS per loan
df_payments['PAID_DATE'] = pd.to_datetime(df_payments['PAID_DATE'])
latest_status = df_payments.sort_values('PAID_DATE').groupby('LOAN_ID').tail(1)[['LOAN_ID', 'STATUS']]

# Step 7: Merge latest status into loan_data
loan_data = loan_data.merge(latest_status, on='LOAN_ID', how='left')

# Step 8: Aggregate customer asset info
customer_assets_agg = df_assets.groupby('CUSTOMER_ID').agg(
    TOTAL_ASSET_WORTH=('ASSET_WORTH', 'sum'),
    NUM_ASSETS=('ASSET_ID', 'count')
).reset_index()

# Step 9: Merge asset data into loan_data
loan_data = loan_data.merge(customer_assets_agg, on='CUSTOMER_ID', how='left')



In [ ]:
# Step 1: Count number of loans per customer
loans_per_customer = df_loans.groupby('CUSTOMER_ID').size().reset_index(name='NUM_LOANS')

# Step 2: Merge into loan_data
loan_data = loan_data.merge(loans_per_customer, on='CUSTOMER_ID', how='left')


In [ ]:
loan_data


In [ ]:
loan_data['TOTAL_PAID'] = loan_data['TOTAL_PAID'].fillna(0)
loan_data['NUM_PAYMENTS'] = loan_data['NUM_PAYMENTS'].fillna(0)
loan_data['STATUS']=loan_data['STATUS'].fillna('NO PAYMENT')

In [ ]:
loan_data

In [ ]:
# Step 1: Define the ordinal columns
cat_cols_ordinal = ['GENDER', 'EMPLOYMENT_STATUS', 'MARITAL_STATUS']

# Step 2: Group by CUSTOMER_ID and take the first occurrence of each ordinal column
unique_customers = loan_data.groupby('CUSTOMER_ID')[cat_cols_ordinal].first().reset_index()

# Step 3: Now perform value_counts on each column using only unique customers
for col in cat_cols_ordinal:
    print(f"\nValue counts for '{col}':")
    print(unique_customers[col].value_counts())


In [ ]:
print(loan_data['LOAN_STATUS'].value_counts(),
loan_data['STATUS'].value_counts())

In [ ]:
# Standardize values in LOAN_STATUS
loan_data['LOAN_STATUS'] = loan_data['LOAN_STATUS'].replace({
    'ONGOING': 'Active'
})

# Standardize values in STATUS
loan_data['STATUS'] = loan_data['STATUS'].replace({
    'Bounced': 'Skipped',
    'ONTIME': 'Paid'
})


In [ ]:
loan_data.dtypes

In [ ]:
num_cols_continous=['AGE','INCOME(MONTH)','CIBIL_SCORE','NUM_PAYMENTS','TOTAL_PAID','TOTAL_ASSET_WORTH']
loan_data[num_cols_continous].describe()

In [ ]:
sns.set(style="whitegrid")

# Define columns
pie_cols = ['GENDER', 'MARITAL_STATUS', 'LOAN_STATUS']
bar_cols = ['EMPLOYMENT_STATUS', 'LOAN_TYPE', 'STATUS']

# Merge all columns to process
cat_cols_ordinal = pie_cols + bar_cols

for col in cat_cols_ordinal:
    if col in ['LOAN_TYPE', 'LOAN_STATUS', 'STATUS']:
        # Handle list-like strings: split and flatten
        expanded = loan_data[col].dropna().apply(lambda x: [i.strip() for i in x.split(',')])
        flat_list = [item for sublist in expanded for item in sublist]
        counts = pd.Series(flat_list).value_counts()
    else:
        # For normal categorical columns
        counts = loan_data[col].dropna().value_counts()
    
    if col in pie_cols:
        # Pie chart
        plt.figure(figsize=(4, 4))
        plt.pie(counts, labels=counts.index, autopct='%1.1f%%', colors=sns.color_palette('Set2'), startangle=140)
        plt.title(f'Distribution of {col} (Pie Chart)')
        plt.axis('equal')
        plt.tight_layout()
        plt.show()
    else:
        # Bar chart
        plt.figure(figsize=(6, 3))
        sns.barplot(x=counts.index, y=counts.values, palette='Set2')
        plt.title(f'Distribution of {col} (Bar Graph)')
        plt.xlabel(col)
        plt.ylabel("Count")
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()


In [ ]:

# 7. Visualizations

# Distribution of AGE
plt.figure(figsize=(8,4))
sns.histplot(loan_data['AGE'].dropna(), bins=30, kde=True)
plt.title("Age Distribution")
plt.show()

# Income distribution
plt.figure(figsize=(8,4))
sns.histplot(loan_data['INCOME(MONTH)'].dropna(), bins=30, kde=True)
plt.title("Income Distribution")
plt.show()

# CIBIL_Score distribution
plt.figure(figsize=(8,4))
sns.histplot(loan_data['CIBIL_SCORE'].dropna(), bins=30, kde=True)
plt.title("CIBIL Score Distribution")
plt.show()

# Bar plot for Gender counts
plt.figure(figsize=(6,4))
sns.countplot(data=loan_data, x='GENDER')
plt.title("Gender Counts")
plt.show()

# Marital Status counts
plt.figure(figsize=(6,4))
sns.countplot(data=loan_data, x='MARITAL_STATUS')
plt.title("Marital Status Counts")
plt.show()

# Employment Status counts
plt.figure(figsize=(6,4))
sns.countplot(data=loan_data, x='EMPLOYMENT_STATUS')
plt.title("Employment Status Counts")
plt.show()

# 8. Correlation heatmap of numeric columns
numeric_cols = [
    "AGE", "INCOME(MONTH)", "CIBIL_SCORE", 
    "NUM_PAYMENTS",  "TOTAL_PAID", "TOTAL_ASSET_WORTH"
]

plt.figure(figsize=(12, 8))
corr = loan_data[numeric_cols].corr()
sns.heatmap(corr, annot=True, fmt=".2f", cmap='coolwarm')
plt.title("Correlation Heatmap")
plt.show()

# 9. Top 5 customers by Total Loan Amount
# print("\nTop 5 customers by Total Loan Amount_paid:")
# print(loan_data.sort_values(by="TOTAL_PAID", ascending=False).head(5))
print("\nTop 5 unique customers by Total Loan Amount Paid:")
top_customers = (
    loan_data.groupby("CUSTOMER_ID")["TOTAL_PAID"]
    .sum()
    .reset_index()
    .sort_values(by="TOTAL_PAID", ascending=False)
    .head(5)
)
print(top_customers)

# 10. Outlier detection for Income and Total_Asset_Worth using boxplots

plt.figure(figsize=(8,4))
sns.boxplot(x=loan_data['INCOME(MONTH)'])
plt.title("Income Boxplot")
plt.show()

plt.figure(figsize=(8,4))
sns.boxplot(x=loan_data['TOTAL_ASSET_WORTH'])
plt.title("Total Asset Worth Boxplot")
plt.show()


In [ ]:
loan_data

CALCULATING DEFAULT_STATUS


In [ ]:
loan_data

In [ ]:
import pandas as pd
from datetime import datetime


# Step 1: Calculate days difference from today
current_date = pd.to_datetime(datetime.today())
loan_data['DAYS_DIFFERENCE'] = (current_date - loan_data['ISSUE_DATE']).dt.days

# Step 2: Convert days to months (approximate)
loan_data['MONTHS'] = round(loan_data['DAYS_DIFFERENCE'] / 30)

# Step 3: Calculate missed months
loan_data['MISSED_MONTHS'] = loan_data['MONTHS'] - loan_data['NUM_PAYMENTS']

# Step 4: Flag = 0 if fully paid, 1 otherwise
loan_data['FLAG'] = loan_data.apply(lambda row: 0 if row['NUM_PAYMENTS'] == row['TERM_MONTHS'] else 1, axis=1)

# Step 5: Determine default status
def determine_status(row):
    if row['FLAG'] == 0:
        return 'Non-Defaulter'
    elif row['MISSED_MONTHS'] > 3:
        return 'Defaulter'
    else:
        return 'Non-Defaulter'

loan_data['DEFAULT_STATUS'] = loan_data.apply(determine_status, axis=1)



In [ ]:
loan_data

In [ ]:
loan_data['DEFAULT_STATUS'].value_counts()

In [ ]:
loan_data['LOAN_STATUS'].value_counts()

Adding to performance_metrics( default_status  per customer  )

In [ ]:

# Group by customer to get aggregated metrics
performance_metrics = loan_data.groupby('CUSTOMER_ID', as_index=False).agg({
    'EMI': 'sum',
    'INCOME(MONTH)': 'first',
    'DEFAULT_STATUS': lambda x: 'Defaulter' if 'Defaulter' in x.values else 'Non-Defaulter'
})

In [ ]:
performance_metrics['DEFAULT_STATUS'].value_counts()

In [ ]:
performance_metrics = loan_data.groupby('CUSTOMER_ID', as_index=False).agg({
    'EMI': 'sum',
    'INCOME(MONTH)': 'first',
    'DEFAULT_STATUS': lambda x: 'Defaulter' if 'Defaulter' in x.values else 'Non-Defaulter'
})

# Step 6: Convert income to numeric
performance_metrics['INCOME(MONTH)'] = pd.to_numeric(performance_metrics['INCOME(MONTH)'], errors='coerce')
performance_metrics['DTI_RATIO'] = performance_metrics['EMI'] / performance_metrics['INCOME(MONTH)']


In [ ]:
performance_metrics['DTI_RATIO'].describe()

In [ ]:

# Step 8: Classify DTI Risk Level
def classify_dti_risk(dti):
    if dti <= 1.0:
        return 'Low'
    elif dti <= 2.0:
        return 'Moderate'
    elif dti <= 3.0:
        return 'Elevated'
    else:
        return 'High'

performance_metrics['DTI_RISK_LEVEL'] = performance_metrics['DTI_RATIO'].apply(classify_dti_risk)

In [ ]:
performance_metrics.dtypes

In [ ]:
performance_metrics['DTI_RISK_LEVEL'].value_counts()

In [ ]:
loan_data

In [ ]:
loan_data['ISSUE_DATE'] = pd.to_datetime(loan_data['ISSUE_DATE']).dt.strftime('%Y-%m-%d')
# Keep ISSUE_DATE as datetime object, do NOT convert to string
loan_data['ISSUE_DATE'] = pd.to_datetime(loan_data['ISSUE_DATE']).dt.date



In [ ]:
loan_data.dtypes

In [ ]:


# Step 3: Write to Snowflake — Snowflake will auto-cast 'YYYY-MM-DD' strings to TIMESTAMP_NTZ
session.write_pandas(
    loan_data,
    table_name="LOAN_SUMMARY",
    database="BANKDATA",
    schema="REPORTING",
    overwrite=True,
)


In [ ]:

# Save to Snowflake
session.write_pandas(
    performance_metrics,
    table_name="PERFORMANCE_METRICS",
    database="BANKDATA",
    schema="REPORTING",
    overwrite=True
)

In [ ]:
print(performance_metrics.columns)


In [ ]:
-- CREATE OR REPLACE PROCEDURE truncate_raw_tables()
-- RETURNS STRING
-- LANGUAGE SQL
-- AS
-- $$
-- BEGIN
--     TRUNCATE TABLE RAW.BANK_CUSTOMERS;
--     TRUNCATE TABLE RAW.BANK_CUSTOMER_ASSETS;  
--     TRUNCATE TABLE RAW.BANK_LOANS;
--     TRUNCATE TABLE RAW.BANK_PAYMENTS;
    
--     RETURN 'All RAW tables truncated.';
-- END;
-- $$;


In [ ]:
-- CALL truncate_raw_tables();
